Experiment 2

In [ ]:
import re

# Define the filename
filename = 'Data_Logs/Experiment_2/Garden/garden_LoS.txt'  # Replace with your actual filename

with open(filename, 'r') as file:
    lines = file.readlines()

# list to store parsed data
parsed_data = []

# Process each line
for line in lines:
    line = line.strip()  # Remove extra whitespace
    if line.startswith("New test."):
        # Extract the parameters using regular expression
        params = re.findall(r'SF: (\d+), CR: (\d+), Tx Power: (\d+), Toa: (\d+)', line)
        if params:
            parsed_data.append({'SF': int(params[0][0]), 'CR': int(params[0][1]), 
                                'Tx Power': int(params[0][2]), 'Toa': int(params[0][3])})

for data in parsed_data:
    print(data)


In [ ]:
# AVE RSSI AND SNR FOR EACH SF
import re
import matplotlib as plt 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def remove_outliers(data, threshold=10):
    # Calculate the mean and standard deviation
    mean = np.mean(data)
    std_dev = np.std(data)
    
    # Filter out points that are further than 'threshold' times the standard deviation
    filtered_data = [x for x in data if abs(x - mean) <= threshold]
    return filtered_data


# Define filename
filename = 'Data_Logs/RSSI_diff_fc/459/RSSI_459_Rx.txt'

with open(filename, 'r') as file:
    lines = file.readlines()

experiments = []

# Initialize a temporary dictionary to store current experiment data
current_experiment = {}

for line in lines:
    line = line.strip()

    # Check if new experiment
    if line.startswith("New test."):
        # Extract parameters using regular expression
        params = re.findall(r'SF: (\d+), CR: (\d+), Tx Power: (\d+), Toa: (\d+)', line)
        if params:
            # If there's already an experiment in progress, save it
            if current_experiment:
                experiments.append(current_experiment)

            # Initialize new experiment data
            current_experiment = {
                'SF': int(params[0][0]),
                'CR': int(params[0][1]),
                'Tx Power': int(params[0][2]),
                'Toa': int(params[0][3]),
                'RSSI': [],
                'SNR': [],
                'AveRSSI':int(params[0][0])
            }
        continue

    # Check for lines with OnRxDone status
    if "OnRxDone" in line:
        # Extract RSSI and SNR using regex
        match = re.match(r'(\w+),OnRxDone,(-?\d+),(-?\d+),\d+,\d+', line)
        if match:
            rssi = int(match.group(2))  # RSSI value
            snr = int(match.group(3))    # SNR value
            
            # Append RSSI and SNR to current experiment's lists
            if current_experiment:  # Ensure we are within a valid experiment
                current_experiment['RSSI'].append(rssi)
                current_experiment['SNR'].append(snr)

# After processing all lines, save the last experiment if it exists
if current_experiment:
    experiments.append(current_experiment)

spreading_factors = []
ave_rssi_vals = []
toa_vals = []
ave_snr_vals = []
stddev_rssi_vals = []
stddev_snr_vals = []

# Print the parsed data
for i, experiment in enumerate(experiments[0:6]):
    rssi_array = np.array(experiment['RSSI']) if experiment['RSSI'] else np.array([0])
    snr_array = np.array(experiment['SNR']) if experiment['SNR'] else np.array([0])

    rssi_array = remove_outliers(rssi_array, 8)
    snr_array = remove_outliers(snr_array, 5)

    avg_rssi = np.mean(rssi_array)
    avg_snr = np.mean(snr_array)

    stddev_rssi = np.std(rssi_array, ddof=1) if len(rssi_array) > 1 else 0
    stddev_snr = np.std(snr_array, ddof=1) if len(snr_array) > 1 else 0


    experiment['AveRSSI'] = avg_rssi
    experiment['AveSNR'] = avg_snr
    experiment['StddevRSSI'] = stddev_rssi
    experiment['StddevSNR'] = stddev_snr

    spreading_factors.append(experiment['SF'])
    ave_rssi_vals.append(experiment['AveRSSI'])
    toa_vals.append(experiment['Toa'])
    ave_snr_vals.append(experiment['AveSNR'])
    stddev_rssi_vals.append(experiment['StddevRSSI'])
    stddev_snr_vals.append(experiment['StddevSNR'])
    print(f"Experiment {i + 1}: SF = {experiment['SF']}, CR = {experiment['CR']}, "
          f"Tx Power = {experiment['Tx Power']}, Toa = {experiment['Toa']}, "
          f"Avg RSSI = {avg_rssi:.2f}, Avg SNR = {avg_snr:.2f}")
    

print(spreading_factors)
print(ave_rssi_vals)
print(ave_snr_vals)
print(toa_vals)

print("RSSI std dev:")
print(stddev_rssi_vals)
print("SNR std dev:")
print(stddev_snr_vals)

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.bar(spreading_factors, ave_rssi_vals, color='blue', label='Average RSSI')
ax1.set_xlabel('Spreading Factor')
ax1.set_ylabel('Average RSSI (dBm)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.plot(spreading_factors, toa_vals, color='red', marker='o', linestyle='-', label='ToA (ms)')
ax2.set_ylabel('ToA per packet (ms)', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('Average RSSI and Time on Air (ToA) for Different Spreading Factors (7-12)')
plt.grid(True)

#SNR
plt.show()

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.bar(spreading_factors, ave_snr_vals, color='blue', label='Average RSSI')
ax1.set_xlabel('Spreading Factor')
ax1.set_ylabel('Average SNR', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.plot(spreading_factors, toa_vals, color='red', marker='o', linestyle='-', label='ToA (ms)')
ax2.set_ylabel('ToA per packet (ms)', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('Average SNR and Time on Air (ToA) for Different Spreading Factors (7-12)')
plt.grid(True)

plt.show()



In [ ]:
# AVE RSSI AND SNR FOR EACH CR
import re
import matplotlib as plt 
import pandas as pd
import matplotlib.pyplot as plt

def remove_outliers(data, threshold=10):
    mean = np.mean(data)
    std_dev = np.std(data)
    
    filtered_data = [x for x in data if abs(x - mean) <= threshold]
    return filtered_data

filename = 'Data_Logs/Experiment_2/CricketField_100m/Rx/ABSORBER_2.txt' 

with open(filename, 'r') as file:
    lines = file.readlines()

# ist to store parsed data for each experiment
experiments = []

# temporary dictionary to store current experiment data
current_experiment = {}

for line in lines:
    line = line.strip() 

    # Check for new experiment
    if line.startswith("New test."):
        # Extract parameters using regular expression
        params = re.findall(r'SF: (\d+), CR: (\d+), Tx Power: (\d+), Toa: (\d+)', line)
        if params:
            # If there's already an experiment in progress, save it
            if current_experiment:
                experiments.append(current_experiment)

            # Initialize new experiment data
            current_experiment = {
                'SF': int(params[0][0]),
                'CR': int(params[0][1]),
                'Tx Power': int(params[0][2]),
                'Toa': int(params[0][3]),
                'RSSI': [],
                'SNR': [],
                'AveRSSI':int(params[0][0])
            }
        continue 

    # Check for lines with OnRxDone status
    if "OnRxDone" in line:
        # Extract RSSI and SNR using regex
        match = re.match(r'(\w+),OnRxDone,(-?\d+),(-?\d+),\d+,\d+', line)
        if match:
            rssi = int(match.group(2))  # RSSI value
            snr = int(match.group(3))    # SNR value
            
            # Append RSSI and SNR to current experiment's lists
            if current_experiment:  # Ensure we are within a valid experiment
                current_experiment['RSSI'].append(rssi)
                current_experiment['SNR'].append(snr)

# After processing all lines, save the last experiment if it exists
if current_experiment:
    experiments.append(current_experiment)

coding_rates = []
ave_rssi_vals = []
toa_vals = []
norm_toa = []
ave_snr_vals = []
stddev_rssi_vals = []
stddev_snr_vals = []

for i, experiment in enumerate(experiments[4:7]):
    rssi_array = np.array(experiment['RSSI']) if experiment['RSSI'] else np.array([0])
    snr_array = np.array(experiment['SNR']) if experiment['SNR'] else np.array([0])

    rssi_array = remove_outliers(rssi_array, 8)
    snr_array = remove_outliers(snr_array, 5)

    avg_rssi = np.mean(rssi_array)
    avg_snr = np.mean(snr_array)

    stddev_rssi = np.std(rssi_array, ddof=1) if len(rssi_array) > 1 else 0
    stddev_snr = np.std(snr_array, ddof=1) if len(snr_array) > 1 else 0


    experiment['AveRSSI'] = avg_rssi
    experiment['AveSNR'] = avg_snr
    experiment['StddevRSSI'] = stddev_rssi
    experiment['StddevSNR'] = stddev_snr

    spreading_factors.append(experiment['SF'])
    ave_rssi_vals.append(experiment['AveRSSI'])
    toa_vals.append(experiment['Toa'])
    ave_snr_vals.append(experiment['AveSNR'])
    stddev_rssi_vals.append(experiment['StddevRSSI'])
    stddev_snr_vals.append(experiment['StddevSNR'])
    print(f"Experiment {i + 1}: SF = {experiment['SF']}, CR = {experiment['CR']}, "
          f"Tx Power = {experiment['Tx Power']}, Toa = {experiment['Toa']}, "
          f"Avg RSSI = {avg_rssi:.2f}, Avg SNR = {avg_snr:.2f}")
    

print(spreading_factors)
print(ave_rssi_vals)
print(ave_snr_vals)
print(toa_vals)

print("RSSI std dev:")
print(stddev_rssi_vals)
print("SNR std dev:")
print(stddev_snr_vals)

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.bar(coding_rates, ave_rssi_vals, color='blue', label='Average RSSI')
ax1.set_xlabel('Coding Rate')
ax1.set_ylabel('Average RSSI (dBm)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.plot(coding_rates, toa_vals, color='red', marker='o', linestyle='-', label='ToA (ms)')
ax2.set_ylabel('ToA per packet (ms)', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('Average RSSI and Time on Air (ToA) for Different Coding Rates (7-12)')
plt.grid(True)

#SNR
plt.show()

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.bar(coding_rates, ave_snr_vals, color='blue', label='Average RSSI')
ax1.set_xlabel('Coding Rate')
ax1.set_ylabel('Average SNR', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.plot(coding_rates, toa_vals, color='red', marker='o', linestyle='-', label='ToA (ms)')
ax2.set_ylabel('ToA per packet (ms)', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('Average SNR and Time on Air (ToA) for Different Coding Rates')
plt.grid(True)
plt.show()



In [ ]:
import re

# Define the filename
filename = 'Data_Logs/Experiment_2/2_RugbyField_400m/Rx/no_occlusion_400m_rx_1.txt'  # Replace with your actual filename

# Open the file
with open(filename, 'r') as file:
    # Read all lines from the file
    lines = file.readlines()

# Initialize a list to store parsed data
parsed_data = []
current_payloads = []  # Initialize a list to store payloads for the current experiment
experiment_started = False  # Flag to check if we've started processing an experiment

# Process each line
for line in lines:
    line = line.strip()  # Remove extra whitespace

    # Detect the start of a new test
    if line.startswith("New test."):
        experiment_started = True  # Set flag to indicate a test has started

        # If a new experiment starts, store the previous experiment's data
        if current_payloads and parsed_data:
            parsed_data[-1]['payloads'] = current_payloads
            current_payloads = []  # Reset for the next experiment

        # Extract parameters using regular expression
        params = re.findall(r'SF: (\d+), CR: (\d+), Tx Power: (\d+), Toa: (\d+)', line)
        if params:
            # Append a new experiment entry with parameters
            parsed_data.append({
                'SF': int(params[0][0]),
                'CR': int(params[0][1]),
                'Tx Power': int(params[0][2]),
                'Toa': int(params[0][3]),
                'payloads': []  # Initialize an empty list for payloads
            })

    # Check for payload lines that start with '0AQ'
    elif experiment_started and line.startswith("0AQ"):  # Ensure we are within an experiment
        # Split the line and convert everything after '0AQ' into integers
        payload_values = [int(i) for i in line.split()[1:] if i.isdigit()]  # Skip the '0AQ'
        if current_payloads is not None:  # Ensure we are in an experiment context
            current_payloads.append(payload_values)  # Add to current payloads

# After processing all lines, store any remaining payloads for the last experiment
if current_payloads and parsed_data:
    parsed_data[-1]['payloads'] = current_payloads

f = open("Payload_Data/400m_no_occlusion.txt", "x")

# Print the parsed data with payloads in space-separated format
for experiment in parsed_data:
    print(f"Experiment with SF: {experiment['SF']}, CR: {experiment['CR']}, Tx Power: {experiment['Tx Power']}, Toa: {experiment['Toa']}:")
    f.write(f"Experiment with SF: {experiment['SF']}, CR: {experiment['CR']}, Tx Power: {experiment['Tx Power']}, Toa: {experiment['Toa']}:\n")
    for payload in experiment['payloads']:
        print(" ".join(map(str, payload)))  # Join the integers in the payload with a space
        f.write(" ".join(map(str, payload)))
        f.write("\n")

f.close()
